## Q4

In [1]:
from itertools import product

# Initial standings as provided
standings = {
    'PHI': {'Wins': 89, 'Losses': 70},
    'ARI': {'Wins': 84, 'Losses': 75},
    'MIA': {'Wins': 82, 'Losses': 76},
    'CHC': {'Wins': 82, 'Losses': 77},
    'CIN': {'Wins': 81, 'Losses': 78},
    'SD': {'Wins': 79, 'Losses': 80}
}

# Each team has 3 remaining games
remaining_games = {team: 3 for team in standings}

# Function to simulate outcomes of the season
def simulate_season(standings, outcomes):
    """
    Purpose: To simulate the season's end based on given outcomes for the remaining games.

    Process:
        A temporary copy of the current standings is made to update the standings without altering the original data.
        For each team, it updates the number of wins and losses based on the simulated outcomes (1 for a win, 0 for a loss).

    Returns: The updated standings after accounting for the outcomes of the remaining games.
    """
    temp_standings = {team: {'Wins': standings[team]['Wins'], 'Losses': standings[team]['Losses']} for team in standings}
    for team, outcome in outcomes.items():
        temp_standings[team]['Wins'] += outcome.count(1)  # Counting wins
        temp_standings[team]['Losses'] += outcome.count(0)  # Counting losses
    return temp_standings

# Function to determine if the suspended game affects Wild Card seeding
def is_game_necessary_for_seeding(final_standings):
    """
    Purpose: To determine if the suspended game is necessary for deciding the final seeding of the Wild Card teams.

    Process:
        - Calculates the winning percentage for each team.
        - Sorts the teams based on their winning percentage, applying tiebreaker rules (San Diego over everyone, Miami over everyone except San Diego).
        - Checks if the outcome of the suspended game could change Miami's position within or into the top three teams.

    Returns: True if the suspended game affects the seeding, False otherwise
    """
    # Calculate the winning percentage for sorting
    for team_stats in final_standings.values():
        team_stats['WinPct'] = team_stats['Wins'] / (team_stats['Wins'] + team_stats['Losses'])

    # Sorting teams based on winning percentage, and applying tiebreaker rules
    sorted_teams = sorted(
        final_standings.items(),
        key=lambda x: (-x[1]['WinPct'], x[0] != 'SD', x[0] == 'MIA')
    )

    # Extracting teams in the playoff positions and their immediate followers
    playoff_teams = sorted_teams[:3]
    fourth_place_team = sorted_teams[3]

    # Check if Miami's game affects their seeding among the top 3 or if they could jump into the top 3 with a win
    for i in range(3):
        if sorted_teams[i][0] == 'MIA' and \
           (sorted_teams[3][1]['WinPct'] >= sorted_teams[i][1]['WinPct'] or \
           (sorted_teams[i][1]['WinPct'] - 1/(sorted_teams[i][1]['Wins'] + sorted_teams[i][1]['Losses'] + 1)) <= sorted_teams[3][1]['WinPct']):
            return True
    if fourth_place_team[0] == 'MIA' and \
       (fourth_place_team[1]['WinPct'] + 1/(fourth_place_team[1]['Wins'] + fourth_place_team[1]['Losses'] + 1)) > playoff_teams[2][1]['WinPct']:
        return True

    return False

# Counting scenarios where the game is necessary for seeding
necessary_game_count_seeding = 0
total_outcomes = 2 ** (3 * len(remaining_games))  # Total number of outcomes

# Iterating over all possible outcomes
for outcome in product([0, 1], repeat=sum(remaining_games.values())):
    """
    This code iterates over every possible combination of wins and losses for the remaining games.

    - product([0, 1], repeat=sum(remaining_games.values())) generates all combinations of game results (0s and 1s) for the remaining games of all teams.

    - outcome_by_team maps each team to their specific game outcomes from the generated combinations.

    - For each combination, it updates the standings using simulate_season and then checks if the suspended game is necessary for seeding with is_game_necessary_for_seeding.
    
    """
    outcome_by_team = {team: outcome[i*3:(i+1)*3] for i, team in enumerate(remaining_games)}
    final_standings = simulate_season(standings, outcome_by_team)
    if is_game_necessary_for_seeding(final_standings):
        necessary_game_count_seeding += 1

# Calculating the probability
probability_seeding = necessary_game_count_seeding / total_outcomes
necessary_game_count_seeding, total_outcomes, probability_seeding


(145216, 262144, 0.553955078125)